In [23]:
import copy
import os 

import numpy as np

from overcooked_ai.overcooked_ai_py.utils import load_pickle, load_pkl, save_pickle, load_dict_from_file, profile

In [33]:
clean_train_trials = load_pkl('human_aware_rl/thesis_data/human/anonymized/clean_train_trials.pkl')
clean_test_trials = load_pkl('human_aware_rl/thesis_data/human/anonymized/clean_test_trials.pkl')
clean_test_trials

,Unnamed: 0,Unnamed: 0.1,cur_gameloop,datetime,is_leader,joint_action,layout,layout_name,next_state,reward,...,state,time_elapsed,time_left,is_wait,completed,run,workerid_num,reward_norm,cur_gameloop_total,reward_norm_total
7215,7215,22170,0.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXXXXXXXX', 'O XSXOX S', 'X P 1 X', 'X2 P...",asymmetric_advantages,"{'players': [{'position': [6, 2], 'orientation...",0.0,...,"{'players': [{'position': [6, 2], 'orientation...",0.151,180.0,True,True,run-1,1,0.0,1204,640.0
7216,7216,22171,1.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXXXXXXXX', 'O XSXOX S', 'X P 1 X', 'X2 P...",asymmetric_advantages,"{'players': [{'position': [6, 2], 'orientation...",0.0,...,"{'players': [{'position': [6, 2], 'orientation...",0.303,180.0,True,True,run-1,1,0.0,1204,640.0
7217,7217,22172,2.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXXXXXXXX', 'O XSXOX S', 'X P 1 X', 'X2 P...",asymmetric_advantages,"{'players': [{'position': [6, 2], 'orientation...",0.0,...,"{'players': [{'position': [6, 2], 'orientation...",0.451,180.0,True,True,run-1,1,0.0,1204,640.0
7218,7218,22173,3.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXXXXXXXX', 'O XSXOX S', 'X P 1 X', 'X2 P...",asymmetric_advantages,"{'players': [{'position': [6, 2], 'orientation...",0.0,...,"{'players': [{'position': [6, 2], 'orientation...",0.601,179.0,True,True,run-1,1,0.0,1204,640.0
7219,7219,22174,4.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXXXXXXXX', 'O XSXOX S', 'X P 1 X', 'X2 P...",asymmetric_advantages,"{'players': [{'position': [6, 2], 'orientation...",0.0,...,"{'players': [{'position': [6, 2], 'orientation...",0.752,179.0,True,True,run-1,1,0.0,1204,640.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136841,136841,174824,1127.0,1.557969e+12,True,"['INTERACT', [0, 0]]","['XXXPPXXX', 'X X', 'D XXXX S', 'X2 1X...",random3,"{'players': [{'position': [3, 1], 'orientation...",0.0,...,"{'players': [{'position': [3, 1], 'orientation...",179.946,0.0,False,True,run-2,22,0.0,1132,220.0
136842,136842,174825,1128.0,1.557969e+12,True,"[[1, 0], [0, 0]]","['XXXPPXXX', 'X X', 'D XXXX S', 'X2 1X...",random3,"{'players': [{'position': [4, 1], 'orientation...",0.0,...,"{'players': [{'position': [3, 1], 'orientation...",180.106,0.0,False,True,run-2,22,0.0,1132,220.0
136843,136843,174826,1129.0,1.557969e+12,True,"[[0, 0], [0, 0]]","['XXXPPXXX', 'X X', 'D XXXX S', 'X2 1X...",random3,"{'players': [{'position': [4, 1], 'orientation...",0.0,...,"{'players': [{'position': [4, 1], 'orientation...",180.267,0.0,True,True,run-2,22,0.0,1132,220.0
136844,136844,174827,1130.0,1.557969e+12,True,"[[1, 0], [-1, 0]]","['XXXPPXXX', 'X X', 'D XXXX S', 'X2 1X...",random3,"{'players': [{'position': [5, 1], 'orientation...",0.0,...,"{'players': [{'position': [4, 1], 'orientation...",180.426,0.0,False,True,run-2,22,0.0,1132,220.0


In [12]:
train_workers = list(clean_train_trials['workerid_num'].unique())
test_workers = list(clean_test_trials['workerid_num'].unique())

In [40]:
PYTHON_LAYOUT_NAME_TO_JS_NAME = {
    "unident_s": "asymmetric_advantages",
    "simple": "cramped_room",
    "random1": "coordination_ring",
    "random0": "random0",
    "random3": "random3"
}

JS_LAYOUT_NAME_TO_PYTHON_NAME = {v:k for k, v in PYTHON_LAYOUT_NAME_TO_JS_NAME.items()}


def separate_trials_data(model):
    """
    Separate the model trial data by human player id.
    """
    clean_trials = load_pkl('human_aware_rl/thesis_data/human/anonymized/clean_{}_trials.pkl'.format(model))
    layouts = list(clean_trials['layout_name'].unique())
    workers = list(clean_trials['workerid_num'].unique())
    
    for layout in layouts:
        print(layout)
        python_layout = JS_LAYOUT_NAME_TO_PYTHON_NAME[layout]
        dir_path = 'human_aware_rl/thesis_data/human/player/{}/'.format(python_layout)
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        for worker in workers:
            current_worker_trials = clean_trials[(clean_trials['layout_name'] == layout) & (clean_trials['workerid_num'] == worker)]
            current_worker_trials.to_pickle(dir_path + 'clean_{}_trials_worker_{}.pkl'.format(model, worker))

In [41]:
separate_trials_data('train')
separate_trials_data('test')

asymmetric_advantages
coordination_ring
cramped_room
random0
random3
asymmetric_advantages
coordination_ring
cramped_room
random0
random3
